**Simple Genetic algorithm for neuroevolution** 

This notebook aims to find a solution to flappy bird by using a simple genetic algorithm to conduct a large search to fit a neural network. The algorithm is based on genomes consisting of a bitstring, mapping the bits to a neural network. 

In [81]:
# imports
%pip install flappy-bird-gym
import numpy as np
import random
import time
import flappy_bird_gym

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\tobia\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Create a structure organism in order to keep track of the chromosome and the fitness

The cell below contains functions for converting the chromosome into a neural network, as well as useful functions like forward pass in order to let the creatures make predictions. 

In [82]:
# DO NOT MODIFY THIS CELL!

def bitstring_to_floats(bitstring, bits_per_value=10, min_val=-2, max_val=2):
    """Converts a bitstring into an array of float values while ensuring full extraction."""
    num_values = len(bitstring) // bits_per_value  # Ensure full number of weights
    if num_values == 0:
        raise ValueError("Bitstring too short for any weights!")

    floats = []
    for i in range(num_values):
        binary_segment = bitstring[i * bits_per_value:(i + 1) * bits_per_value]
        decimal_value = int(binary_segment, 2)  # Convert binary to decimal
        scaled_value = min_val + (max_val - min_val) * (decimal_value / (2**bits_per_value - 1))  # Normalize to [-2,2]
        floats.append(scaled_value)

    return np.array(floats)


def decode_architecture(bitstring, bits_per_weight=10, input_size=2, output_size=1):
    """Extracts NN structure (neurons) and weights from a bitstring"""
    hidden_neurons = int(bitstring[:4], 2) + 1  # Allow 1-16 hidden neurons
    weights = bitstring_to_floats(bitstring[4:], bits_per_value=bits_per_weight)

    # Compute expected number of weights
    required_params = (input_size * hidden_neurons) + hidden_neurons + (hidden_neurons * output_size) + output_size

    if len(weights) < required_params:
        raise ValueError(f"Decoded weights ({len(weights)}) are fewer than expected ({required_params}). "
                         f"Ensure bitstring is at least {required_params * bits_per_weight} bits long!")

    return hidden_neurons, weights


def construct_nn(bitstring, input_size=2, output_size=1):
    """Constructs a simple 1-layer NN from a bitstring with dynamic hidden neurons"""
    hidden_neurons, params = decode_architecture(bitstring, input_size=input_size, output_size=output_size)

    # Compute parameter indices
    w1_end = input_size * hidden_neurons
    b1_end = w1_end + hidden_neurons
    w2_end = b1_end + (hidden_neurons * output_size)

    # Debugging: Print out parameter sizes
    # print(f"Params Length: {len(params)} | Expected: {w2_end + output_size}")
    # print(f"Hidden Neurons: {hidden_neurons}, Input Size: {input_size}, Output Size: {output_size}")
    # print(f"w1: (0:{w1_end}), b1: ({w1_end}:{b1_end}), w2: ({b1_end}:{w2_end}), b2: ({w2_end}:{w2_end + output_size})")

    # Extract weights and biases
    w1 = params[:w1_end].reshape(input_size, hidden_neurons)  # (input_size, hidden_neurons)
    b1 = params[w1_end:b1_end]  # (hidden_neurons,)
    w2 = params[b1_end:w2_end].reshape(hidden_neurons, output_size)  # (hidden_neurons, output_size)
    b2 = params[w2_end:w2_end + output_size]  # (output_size,)

    return w1, b1, w2, b2



def forward_pass(X, w1, b1, w2, b2):
    """Performs forward propagation through the neural network"""
    hidden = np.tanh(np.dot(X, w1) + b1)  # (batch, hidden)
    output = np.tanh(np.dot(hidden, w2) + b2)  # (batch, 1)

    return output

def required_bitstring_length(hidden_neurons=16, input_size=2, output_size=1, bits_per_weight=10):
    """Compute the required bitstring length for encoding all NN parameters."""
    required_params = (input_size * hidden_neurons) + hidden_neurons + (hidden_neurons * output_size) + output_size
    return required_params * bits_per_weight  # Convert to bitstring length







In [83]:
# Save and load best genome of a run
def save_best_genome(genome, filename):
    """Saves the best genome to a file"""
    
    with open (filename, 'w') as f:
        f.write(genome)
        
def load_best_genome(filename):
    """Loads the best genome from a file"""
    
    with open (filename, 'r') as f:
        genome = f.read()
        
    return genome

**Initialize population**

In [84]:
def initialize_population(pop_size, genome_length):
    """Initializes a population of genomes"""
    return["".join(np.random.choice(["0", "1"], genome_length)) for _ in range(pop_size)]


In [85]:
# TEST
def test_initialize_population():
    np.random.seed(42)
    pop_size = 3
    genome_length = 5
    population = initialize_population(pop_size, genome_length)
    
    expected_population = ['01000', '10001', '00001']
    
    assert len(population) == pop_size, f"Expected population size {pop_size}, got {len(population)}"
    
    # Verify each genome has the correct length and contains only '0' and '1'
    for genome in population:
        assert len(genome) == genome_length, f"Expected genome length {genome_length}, got {len(genome)}"
        for char in genome:
            assert char in ['0', '1'], f"Unexpected character {char} in genome"

    assert population == expected_population, f"Expected {expected_population}, got {population}"
    print("All tests passed!")

test_initialize_population()

All tests passed!


**Fitness Function**
The fitness of a genome is calculated in a fitness function. This often the only domain centric part of a genetic algorithm, meaning that the neural network will be trained based on what kind of environment the fitness function uses.

In [86]:


env = flappy_bird_gym.make("FlappyBird-v0")

def evaluate_fitness(chromosome):
    """Evaluates a chromosome based on its fitness."""
    hidden_neurons, params = decode_architecture(chromosome)
    w1, b1, w2, b2 = construct_nn(chromosome)

    # Run the game
    obs, _ = env.reset()
    total_reward = 0
    while True:
        
        try:
            X = np.array(obs).reshape(1, 2)   
        except:
            X = np.array([0.1,1.0]).reshape(1, 2) # the first observation is not in the correct shape

        # Forward pass through the NN
        output = forward_pass(X, w1, b1, w2, b2)
        
        # TODO: play around with the threshold and see if you can get a better model
        action = 1 if output > 0.7 else 0

        # Take the action
        obs, reward, terminated, info = env.step(action)
        total_reward += reward - action * 7
        
        # Check if the game is over
        if terminated:
            break
        
    # clamp reward to be at least 0.000001
    return max(total_reward, 0.000001) +  + int(info['score'])*1000



In [87]:
# TEST 
def test_evaluate_fitness():
    np.random.seed(42)
    dummy_chromosome = "1011000110"*50
    fitness = evaluate_fitness(dummy_chromosome)
    assert isinstance(fitness, (int, float)), f"Expected fitness to be a number, got {type(fitness)}"
    assert fitness >= 0, f"Expected fitness to be non-negative, got {fitness}"
    print("Test passed!")

test_evaluate_fitness()

Test passed!


**Crossover**
The crossover function decides how different genomes should mate in order to produce offspring. 
The gene representation is in the form of a bitstring.

**Task** 
Define your own set of crossover functions by implementing different policies to combine the bitstirng of two parents. All genomes are the same length. 

In [88]:
def uniform_crossover(parent1, parent2):
    """Uniform crossover for binary genomes"""
    mask = np.random.randint(2, size=len(parent1))  # Random bit mask
    child1 = "".join([parent1[i] if mask[i] else parent2[i] for i in range(len(parent1))])
    child2 = "".join([parent2[i] if mask[i] else parent1[i] for i in range(len(parent1))])
    return child1, child2


def single_point_crossover(parent1, parent2, crossover_rate=0.9):
    """Single-point crossover for binary genomes"""
    if np.random.rand() > crossover_rate:
        return parent1, parent2
    split_point = np.random.randint(1, len(parent1) - 1)  # Random split point
    child1 = parent1[:split_point] + parent2[split_point:]
    child2 = parent2[:split_point] + parent1[split_point:]
    return child1, child2


def crossover(parent1, parent2, crossover_rate=0.9):
    random_number = np.random.rand()
    if random_number < 0.5:
        return single_point_crossover(parent1, parent2, crossover_rate)
    else:
        return uniform_crossover(parent1, parent2)



In [ ]:
# Test for uniform_crossover
def test_uniform_crossover():
    try:
        np.random.seed(42)
        # Define parent genomes.
        parent1 = "11111"
        parent2 = "00000"
    
        expected_child1 = "01000"
        expected_child2 = "10111"

        child1, child2 = uniform_crossover(parent1, parent2)
        assert child1 == expected_child1, f"Expected child1 {expected_child1}, got {child1}"
        assert child2 == expected_child2, f"Expected child2 {expected_child2}, got {child2}"
        print("uniform_crossover passed!")
    except NameError:
        print("uniform_crossover is not implemented.")
    except Exception as e:
        print("uniform_crossover failed:", e)

# Test for single_point_crossover
def test_single_point_crossover():
    try:
        np.random.seed(42)
        # Define parent genomes.
        parent1 = "11111"
        parent2 = "00000"

        expected_child1 = "10000"
        expected_child2 = "01111"

        child1, child2 = single_point_crossover(parent1, parent2)
        assert child1 == expected_child1, f"Expected child1 {expected_child1}, got {child1}"
        assert child2 == expected_child2, f"Expected child2 {expected_child2}, got {child2}"
        print("single_point_crossover passed!")
    except NameError:
        print("single_point_crossover is not implemented.")
    except Exception as e:
        print("single_point_crossover failed:", e)

def run_crossover_tests():
    print("Running crossover tests...\n")
    test_uniform_crossover()
    test_single_point_crossover()
    print("\nIf at least one of the tests passed, the implementation is considered acceptable.")

run_crossover_tests()

Running crossover tests...

uniform_crossover passed!
single_point_crossover passed!

If at least one of the tests passed, the implementation is considered acceptable.


In [90]:
def bit_flip_mutation(bitstring, mutation_rate=0.01):
    """Flips random bits with given probability"""
    return "".join([bit if np.random.rand() > mutation_rate else str(1 - int(bit)) for bit in bitstring])


def adaptive_mutate(chromosome, generation, max_generations, mutation_rate=0.01, diversity_factor=1):
    """Adapts mutation rate based on population diversity."""
    adjusted_mutation_rate = mutation_rate * (1 - (generation / max_generations)) * diversity_factor

    chromosome_list = list(chromosome)  # Convert to mutable list
    for i in range(len(chromosome_list)):
        if np.random.rand() < adjusted_mutation_rate:
            chromosome_list[i] = '1' if chromosome_list[i] == '0' else '0'  # Flip bit

    return "".join(chromosome_list)  # Convert back to string


def swap_mutation(chromosome):
    """Performs swap mutation on a given chromosome by swapping two random genes."""
    if len(chromosome) < 2:
        return chromosome  # No mutation possible
    
    idx1, idx2 = random.sample(range(len(chromosome)), 2)
    chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]
    
    return chromosome



In [91]:
def deterministic_crowding(parent1, parent2, child1, child2, fitness_function):
    """Ensures offspring compete with similar parents."""
    f_p1 = fitness_function(parent1)
    f_p2 = fitness_function(parent2)
    f_c1 = fitness_function(child1)
    f_c2 = fitness_function(child2)

    new1 = child1 if f_c1 > f_p1 else parent1
    new2 = child2 if f_c2 > f_p2 else parent2

    return new1, new2


In [92]:
def hamming_distance(bitstring1, bitstring2):
    """Calculates Hamming distance between two bitstrings."""
    return sum(b1 != b2 for b1, b2 in zip(bitstring1, bitstring2))


def fitness_sharing(population, fitnesses, sigma_share=10, alpha=2):
    """Applies fitness sharing to promote diversity."""
    shared_fitnesses = np.zeros(len(fitnesses))

    for i in range(len(population)):
        niche_count = 0
        for j in range(len(population)):
            if i != j:
                distance = hamming_distance(population[i], population[j])
                if distance < sigma_share:  # If genomes are similar
                    niche_count += (1 - (distance / sigma_share)) ** alpha

        shared_fitnesses[i] = fitnesses[i] / (1 + niche_count)  # Penalize common solutions

    return shared_fitnesses


In [93]:
def tournament_selection(population, fitnesses, tournament_size=10):
    """Select best genome from a random subset"""
    indices = np.random.choice(len(population), tournament_size, replace=False)
    best_index = indices[np.argmax([fitnesses[i] for i in indices])]
    return population[best_index]


def roulette_wheel_selection(population, fitnesses):
    """Selects individuals using fitness proportionate selection."""
    total_fitness = np.sum(fitnesses)
    probabilities = fitnesses / total_fitness
    return population[np.random.choice(len(population), p=probabilities)]


def rank_based_selection(population):
    """Selects a parent based on rank selection."""
    population_sorted = sorted(population, key=lambda org: org.fitness)
    ranks = np.arange(1, len(population) + 1)
    probabilities = ranks / ranks.sum()
    return np.random.choice(population_sorted, p=probabilities)


In [94]:
import numpy as np

# Test for tournament_selection
def test_tournament_selection():
    try:
        np.random.seed(42)
        population = ['A', 'B', 'C', 'D', 'E']
        fitnesses = [10, 20, 30, 40, 50]
        result = tournament_selection(population, fitnesses, tournament_size=3)
        # then the best fitness among these is fitness 50 at index 4 -> population element 'E'
        expected = 'E'
        assert result == expected, f"Expected {expected}, got {result}"
        print("tournament_selection passed!")
    except NameError:
        print("tournament_selection is not implemented.")
    except Exception as e:
        print(f"tournament_selection failed: {e}")

# Test for roulette_wheel_selection
def test_roulette_wheel_selection():
    try:
        np.random.seed(42)
        population = ['A', 'B', 'C']
        fitnesses = np.array([1, 2, 3])  # Total fitness = 6; probabilities: [0.1667, 0.3333, 0.5]
        
        # With seed 42, np.random.choice should reproducibly select index 1 (if the first random number is ~0.3745).
        result = roulette_wheel_selection(population, fitnesses)
        expected = 'B'
        assert result == expected, f"Expected {expected}, got {result}"
        print("roulette_wheel_selection passed!")
    except NameError:
        print("roulette_wheel_selection is not implemented.")
    except Exception as e:
        print(f"roulette_wheel_selection failed: {e}")

# Test for rank_based_selection
def test_rank_based_selection():
    try:
        np.random.seed(42)
        # Create a dummy organism class with a 'fitness' attribute.
        class Organism:
            def __init__(self, name, fitness):
                self.name = name
                self.fitness = fitness
            def __repr__(self):
                return f"Organism({self.name}, {self.fitness})"
        
        # Create a population of organisms.
        population = [
            Organism("A", 10),
            Organism("B", 20),
            Organism("C", 30)
        ]
        result = rank_based_selection(population)
        expected_name = "B"
        assert result.name == expected_name, f"Expected organism {expected_name}, got {result.name}"
        print("rank_based_selection passed!")
    except NameError:
        print("rank_based_selection is not implemented.")
    except Exception as e:
        print(f"rank_based_selection failed: {e}")

def run_selection_tests():
    print("Running selection tests...\n")
    test_tournament_selection()
    test_roulette_wheel_selection()
    test_rank_based_selection()
    print("\nIf at least one of the tests passed, the implementation is considered acceptable.")

run_selection_tests()

Running selection tests...

tournament_selection passed!
roulette_wheel_selection passed!
rank_based_selection passed!

If at least one of the tests passed, the implementation is considered acceptable.


In [95]:
def genetic_algorithm(pop_size, genome_length, generations):
    """Runs a genetic algorithm to evolve a neural network"""
    # Initialize population (random bitstrings)
    population = initialize_population(pop_size, genome_length)
    max_generation = population

    for gen in range(generations):
        fitnesses = np.array([evaluate_fitness(ind) for ind in population])
        fitnesses = fitness_sharing(population, fitnesses)  # Apply fitness sharing
        
        new_population = []
        # TODO: Elitism saves the top 20 % of the population. Play around with this number and see if you can get a better model
        # keep the 20% best genomes
        new_population.extend([population[i] for i in np.argsort(fitnesses)[-int(pop_size * 0.2):]])
        while len(new_population) < pop_size:
            # Select parents
            #parent1, parent2 = tournament_selection(population, fitnesses), tournament_selection(population, fitnesses)
            parent1 = roulette_wheel_selection(population, fitnesses)
            parent2 = roulette_wheel_selection(population, fitnesses)
            # Crossover & Mutation
            #child1, child2 = uniform_crossover(parent1, parent2)
            child1, child2 = crossover(parent1, parent2)
            child1, child2 = bit_flip_mutation(child1), bit_flip_mutation(child2)
            #diversity_factor = 1 + (np.mean(fitness_sharing(population, fitnesses)) * 0.5)  # Scale mutation if diversity is low
            #child1 = adaptive_mutate(child1, gen, generations, diversity_factor=diversity_factor)
            #child2 = adaptive_mutate(child2, gen, generations, diversity_factor=diversity_factor)
            #child1, child2 = deterministic_crowding(parent1, parent2, child1, child2, evaluate_fitness)
            
            new_population.extend([child1, child2])

        # Replace population (keep best elite)
        best_idx = np.argmax(fitnesses)
        best_genome = population[best_idx]
        population = new_population[:pop_size]
        population[0] = best_genome  # Elitism
        if population[np.argmax(fitnesses)] == best_genome:
            max_generation = population.copy()
        
        print(f"Generation {gen + 1}: Best Fitness = {max(fitnesses):.6f} avg fitness = {np.mean(fitnesses):.6f}")
       
    
    best_genome = max_generation[np.argmax(fitnesses)]
    print(f"Best genome: {best_genome}")
    return best_genome  # Return best genome


In [96]:
random_bitstring = "".join(np.random.choice(["0", "1"], required_bitstring_length(hidden_neurons=11)))

try:
    w1, b1, w2, b2 = construct_nn(random_bitstring)
    print("Neural Network successfully built with shapes:")
    print(f"w1: {w1.shape}, b1: {b1.shape}, w2: {w2.shape}, b2: {b2.shape}")
except ValueError as e:
    print("Error:", e)


input_size = 2
output_size = 1
hidden_neurons = 16  # Max number of hidden neurons
bits_per_weight = 10

required_length = required_bitstring_length(hidden_neurons, input_size, output_size, bits_per_weight) +10

print(f"Required bitstring length: {required_length} bits")  # Debugging

best_genome = genetic_algorithm(
    pop_size=50, 
    genome_length=required_length,  # 16 parameters × 8 bits
    generations=50
    
)
save_best_genome(best_genome, "best_genome.txt")






Neural Network successfully built with shapes:
w1: (2, 2), b1: (2,), w2: (2, 1), b2: (1,)
Required bitstring length: 660 bits
Generation 1: Best Fitness = 59.000000 avg fitness = 20.000000
Generation 2: Best Fitness = 1162.790698 avg fitness = 48.521237
Generation 3: Best Fitness = 2777.777779 avg fitness = 132.025901
Generation 4: Best Fitness = 3765.671642 avg fitness = 238.416724
Generation 5: Best Fitness = 1080.000000 avg fitness = 317.309628
Generation 6: Best Fitness = 13027.000000 avg fitness = 551.587870
Generation 7: Best Fitness = 16286.868687 avg fitness = 1080.640235
Generation 8: Best Fitness = 18430.201342 avg fitness = 1485.736056
Generation 9: Best Fitness = 361327.000000 avg fitness = 18677.738574
Generation 10: Best Fitness = 276417.000000 avg fitness = 17701.249568
Generation 11: Best Fitness = 195245.145631 avg fitness = 20621.214512


KeyboardInterrupt: 

In [ ]:

import random
env = flappy_bird_gym.make("FlappyBird-v0")

num_inputs = 2 # The envirement has 12 observations for each frame
num_outputs = 1 # The envirement has 1 action space (flap or do nothing)
    
best_genome = load_best_genome("best_genome_score_275.txt")
w1, b1, w2, b2 = construct_nn(best_genome, input_size=num_inputs, output_size=num_outputs)
total_reward = 0
best_score = 0

for i in range(100):
    obs, _ = env.reset()
    while True:
        try:
            X = np.array(obs).reshape(1, 2)
        except:
            X = np.array([0.1,1.0]).reshape(1, 2)
        # Forward pass through the NN
        output = forward_pass(X, w1, b1, w2, b2)
        action = 1 if output > 0.7 else 0
        obs, reward, terminated, info = env.step(action)
        
        if terminated:
            break
    print(info)
    if info['score'] > best_score:
        best_score = info['score']
    
print("best score: ", best_score)   

env.close()





{'score': 79}
{'score': 1}
{'score': 157}
{'score': 23}
{'score': 19}
{'score': 114}
{'score': 27}
{'score': 13}
{'score': 16}
{'score': 91}
{'score': 41}
{'score': 41}
{'score': 168}
{'score': 11}
{'score': 184}
{'score': 128}
{'score': 329}
{'score': 187}
{'score': 18}
{'score': 395}
{'score': 178}
{'score': 267}
{'score': 240}
{'score': 61}
{'score': 207}
{'score': 127}
{'score': 61}
{'score': 127}
{'score': 40}
{'score': 453}
{'score': 62}
{'score': 111}
{'score': 152}
{'score': 183}
{'score': 11}
{'score': 66}
{'score': 65}
{'score': 31}
{'score': 25}
{'score': 75}
{'score': 89}
{'score': 2}
{'score': 3}
{'score': 73}
{'score': 103}
{'score': 15}
{'score': 444}
{'score': 31}
{'score': 25}
{'score': 231}
{'score': 89}
{'score': 39}
{'score': 55}
{'score': 314}
{'score': 185}
{'score': 155}
{'score': 187}
{'score': 7}
{'score': 48}
{'score': 402}
{'score': 119}
{'score': 33}
{'score': 103}
{'score': 80}
{'score': 22}
{'score': 157}
{'score': 126}
{'score': 55}
{'score': 39}
{'score'

In [ ]:

import random
env = flappy_bird_gym.make("FlappyBird-v0")


# ===============================================================
# Neural network parameters:
# ===============================================================
num_inputs = 2 # The envirement has 12 observations for each frame
num_outputs = 1 # The envirement has 1 action space (flap or do nothing)
    
best_genome = load_best_genome("best_genome_score_275.txt")
w1, b1, w2, b2 = construct_nn(best_genome, input_size=num_inputs, output_size=num_outputs)
obs, _ = env.reset()
total_reward = 0
while True:
    
    
    try:
        X = np.array(obs).reshape(1, 2)
    except:
        X = np.array([0.1,1.0]).reshape(1, 2)

    # Forward pass through the NN
    output = forward_pass(X, w1, b1, w2, b2)
    action = 1 if output > 0.7 else 0
    obs, reward, terminated, info = env.step(action)
    total_reward += reward
    
    env.render()
    time.sleep(1 / 30)
    
    if terminated:
        break
print(info)
    
  

env.close()





KeyboardInterrupt: 

In [ ]:
import pygame
import numpy as np

def draw_neural_network(chromosome, screen_size=(800, 600)):
    """
    Draws a neural network based on the given chromosome using Pygame.

    Parameters:
    - chromosome: Bitstring representing the neural network structure.
    - screen_size: Tuple (width, height) for pygame window.
    """
    # Decode the architecture and construct the neural network
    hidden_neurons, params = decode_architecture(chromosome)
    w1, b1, w2, b2 = construct_nn(chromosome)

    # Ensure hidden_neurons is a list
    input_neurons = 2  # From the Flappy Bird environment
    output_neurons = 1  # Single output neuron
    layers = [input_neurons] + ([hidden_neurons] if isinstance(hidden_neurons, int) else hidden_neurons) + [output_neurons]

    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    pygame.display.set_caption("Neural Network Visualization")

    # Colors
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    BLUE = (50, 150, 255)
    GREEN = (100, 255, 100)
    RED = (255, 50, 50)

    screen.fill(WHITE)

    # Define spacing
    width, height = screen_size
    layer_spacing = width // (len(layers) + 1)  # Space between layers
    node_radius = 20  # Neuron size
    y_padding = 80  # Padding from top/bottom

    # Compute node positions
    node_positions = []  # List of lists for neuron positions

    for i, neurons in enumerate(layers):
        x = (i + 1) * layer_spacing
        y_spacing = (height - 2 * y_padding) // (neurons - 1) if neurons > 1 else 200

        layer_nodes = []
        for j in range(neurons):
            y = y_padding + j * y_spacing
            layer_nodes.append((x, y))

        node_positions.append(layer_nodes)

    # Draw connections (edges) with weight intensity
    for i in range(len(layers) - 1):
        weight_matrix = w1 if i == 0 else w2  # Use w1 for input-hidden, w2 for hidden-output
        layer1_nodes = node_positions[i]
        layer2_nodes = node_positions[i + 1]

        for j, node1 in enumerate(layer1_nodes):  # Nodes in layer i
            for k, node2 in enumerate(layer2_nodes):  # Nodes in layer i+1
                try:
                    weight = weight_matrix[k, j]  # Extract weight
                    color = GREEN if weight > 0 else RED  # Green for positive, red for negative
                    thickness = int(abs(weight) * 5) + 1  # Scale thickness
                except:
                    pass
                    
                pygame.draw.line(screen, color, node1, node2, thickness)

    # Draw nodes (neurons)
    for layer in node_positions:
        for node in layer:
            
            pygame.draw.circle(screen, BLUE, node, node_radius)
            pygame.draw.circle(screen, BLACK, node, node_radius, 2)  # Outline

    # Display network
    pygame.display.flip()

    # Wait until user closes the window
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.quit()



draw_neural_network(load_best_genome("best_genome.txt"))